In [12]:
from time import time
R.<x> = ZZ[];

class NTRUEncrypt(object):

    def sample(self,NN, o,mo):
        s=[1]*o+[-1]*mo+[0]*(NN-o-mo)
        shuffle(s)
        return R(s)

    def __pn__(self,d1,d2,d3):
        a=ZZ.random_element(2*d1, self.n-d2)
        p1=self.sample(a, d1, d1)
        p2=self.sample(self.n-a, d2, d2)
        p3=self.sample(self.n, d3, d3)
        poly=p1*p2+p3
        return poly

    #reduces the coefficients of polynomial f modulo pp
    #and "fixes" them so that they belong to [-pp/2,pp/2]
    def modCoeffs(self,f,pp):
        clist=f.list()
        p2=pp/2
        for i in range(len(clist)):
            clist[i] = clist[i]%pp
            if clist[i]>p2:
                clist[i]-=pp
        return R(clist)

    def encrypt(self,h,m):
        s=self.sample(self.n, self.Dm, self.Dm-1)
        c=s*h+m
        c=c%(x^self.n-1)
        c=self.modCoeffs(c,self.q)
        return c

    def decrypt(self,c,Priv):
        f,fp=Priv
        a=f*c
        a=a%(x^self.n-1)
        a=self.modCoeffs(a,self.q)
        a=a*fp
        a=a%(x^self.n-1)
        a=self.modCoeffs(a,self.p)
        return a

    def __inv_poly_mod2__(self,poly):
        k=0;b=1;c=0*x;
        f=poly;g=x^self.n-1
        f=self.modCoeffs(f, 2)
        res=False
        while True:
            while f(0)==0 and not f.is_zero():
                f=f.shift(-1)
                c=c.shift(1)
                c=self.modCoeffs(c, 2)
                k+=1
            if f.is_one():
                e=(-k)%self.n
                retval= x^e*b 
                res=True
                break
            elif f.degree()==-1 or f.is_zero():
                break
            if f.degree()<g.degree():
                f,g=g,f
                b,c=c,b
            f=f+g
            b=b+c
            f=self.modCoeffs(f, 2)
            c=self.modCoeffs(c, 2)
        if res:
            retval=retval%(x^self.n-1)
            retval=self.modCoeffs(retval, 2)
            return True, retval
        else:
            return False,0

    def __inv_poly_mod3__(self,poly):
        k=0;b=1;c=0*x;
        f=poly;g=x^self.n-1
        res=False
        while True:
            while f(0)==0 and not f.is_zero():
                f=f.shift(-1)
                c=c.shift(1)
                k+=1
            if f.is_one():
                e=(-k)%self.n
                retval= x^e*b 
                res=True
                break
            elif (-f).is_one():
                e=(-k)%self.n
                retval= -x^e*b 
                res=True
                break
            elif f.degree()==-1 or f.is_zero():
                break
            if f.degree()<g.degree():
                f,g=g,f
                b,c=c,b
            if f(0)==g(0):
                f=f-g
                b=b-c
            else:
                f=f+g
                b=b+c 
            f=self.modCoeffs(f, 3)
            c=self.modCoeffs(c, 3)
        if res:
            retval=retval%(x^self.n-1)
            retval=self.modCoeffs(retval, 3)
            return True, retval
        else:
            return False,0

    def __inv_poly_mod_prime_pow__(self,poly):
        res,b=self.__inv_poly_mod2__(poly)
        if res:
            qr=2
            while qr<self.q:
                qr=qr^2
                b=b*(2-poly*b)
                b=b%(x^self.n-1)
                b=self.modCoeffs(b, self.q)
            return True,b
        else:
            return False,0

    def __gen_priv_key__(self):
        res=False
        while (res==False):
            poly=self.__pn__(self.D1,self.D2,self.D3)
            poly=1+2*poly
            print (poly)
            print ("==poly==")
            ppInv=self.__inv_poly_mod3__(poly)[1]
            print ("==inv3==")
            res,pqInv=self.__inv_poly_mod_prime_pow__(poly)
            print ("==invq==")
        return poly,ppInv,pqInv

    def gen_keys(self):
        f,fp,fq=self.__gen_priv_key__()
        g=self.sample(n,self.Dg,self.Dg-1)
        h=self.p*g*fq
        h=h%(x^self.n-1)
        h=self.modCoeffs(h,self.q)
        return h,(f,fp)

    def __init__(self, SECLEVEL):
        self.p=3
        self.q=2048
        if SECLEVEL==128:
            self.n=439
            self.D1=9;self.D2=8;self.D3=5
            self.Dg=146;self.Dm=112
        elif SECLEVEL==192:
            self.n=593
            self.D1=10;self.D2=10;self.D3=8
            self.Dg=197;self.Dm=158
        else:
            self.n=743
            self.D1=11;self.D2=11;self.D3=15
            self.Dg=247;self.Dm=204
for i in range(10):
    ntru=NTRUEncrypt(256)
    n=ntru.n; dm=ntru.Dm
    m=ntru.sample(n,dm,dm-1)
    ts=time()
    h,Priv=ntru.gen_keys()
    kgtime= time()-ts
    ts=time()
    cc=ntru.encrypt(h,m)
    enctime=time()-ts
    ts=time()
    mm=ntru.decrypt(cc,Priv)
    dectime=time()-ts
    print kgtime,enctime,dectime
    print mm==m

    m1=ntru.sample(n,dm,dm-1)
    m2=ntru.sample(n,dm,dm-1)
    cc1=ntru.encrypt(h,m1)
    cc2=ntru.encrypt(h,m2)

    mm=ntru.decrypt(cc1+cc2,Priv)
    mmm=ntru.modCoeffs(m1+m2,ntru.p)
    print mm==mmm

-2*x^725 + 2*x^705 + 2*x^701 + 2*x^698 + 2*x^694 - 2*x^688 - 2*x^679 - 2*x^666 + 2*x^655 + 2*x^640 + 2*x^633 - 2*x^625 - 2*x^623 + 2*x^619 + 2*x^618 + 2*x^611 + 2*x^607 + 2*x^604 + 2*x^600 - 2*x^596 + 4*x^595 - 4*x^594 + 2*x^590 - 2*x^585 - 2*x^579 - 2*x^578 + 2*x^577 - 4*x^573 - 2*x^571 + 2*x^569 - 2*x^567 - 2*x^564 + 2*x^561 - 2*x^560 + 2*x^554 + 2*x^553 + 2*x^549 - 2*x^545 - 2*x^544 - 2*x^542 - 2*x^541 - 4*x^539 - 2*x^538 - 4*x^535 - 2*x^534 - 4*x^531 + 2*x^530 + 2*x^528 + 4*x^525 + 2*x^524 + 2*x^522 + 2*x^521 + 2*x^520 + 2*x^519 - 2*x^515 - 2*x^514 + 2*x^512 + 2*x^509 - 2*x^508 - 2*x^506 + 2*x^504 - 4*x^502 + 2*x^501 + 2*x^491 - 2*x^489 + 2*x^488 - 2*x^485 + 4*x^483 + 2*x^482 + 4*x^476 + 2*x^475 - 2*x^473 + 4*x^472 + 2*x^470 - 2*x^466 - 2*x^463 - 2*x^461 - 2*x^458 - 4*x^457 - 2*x^456 - 4*x^454 - 2*x^453 - 2*x^452 - 2*x^450 + 2*x^446 + 6*x^445 + 2*x^444 + 6*x^441 + 2*x^436 - 2*x^435 + 2*x^433 + 2*x^430 - 2*x^429 - 2*x^428 - 4*x^426 - 2*x^423 + 2*x^422 + 2*x^418 - 2*x^417 - 2*x^416 -

==inv3==
==invq==
0.60597205162 0.00121188163757 0.00139403343201
True
True
2*x^717 + 2*x^710 + 2*x^707 + 2*x^700 + 2*x^697 - 2*x^695 + 2*x^692 - 2*x^689 - 2*x^688 - 2*x^684 - 2*x^682 - 2*x^681 + 2*x^677 + 2*x^676 + 2*x^675 + 2*x^674 - 2*x^672 - 2*x^669 - 2*x^666 + 2*x^665 - 2*x^664 + 2*x^663 - 2*x^661 - 2*x^658 + 2*x^655 + 2*x^652 - 2*x^651 - 4*x^649 + 2*x^648 + 2*x^644 + 4*x^643 + 2*x^642 + 2*x^641 + 2*x^640 + 2*x^636 - 4*x^634 - 2*x^632 - 2*x^629 - 2*x^627 - 6*x^622 + 2*x^620 - 2*x^619 + 2*x^616 - 2*x^615 - 4*x^614 - 2*x^611 + 2*x^608 + 2*x^606 - 2*x^604 - 2*x^599 - 2*x^597 + 2*x^594 + 2*x^593 + 2*x^591 + 2*x^588 - 2*x^587 + 4*x^586 - 2*x^585 - 2*x^583 - 2*x^582 - 2*x^581 - 2*x^579 + 2*x^578 - 2*x^577 + 2*x^576 + 2*x^575 + 2*x^573 - 2*x^572 - 2*x^570 - 2*x^567 - 2*x^565 + 2*x^564 + 2*x^563 - 2*x^562 + 2*x^561 + 2*x^560 - 2*x^559 + 2*x^557 - 2*x^553 + 2*x^552 + 2*x^551 - 4*x^550 + 4*x^549 - 4*x^548 - 2*x^547 - 2*x^545 + 4*x^544 + 2*x^543 + 4*x^542 + 2*x^540 + 2*x^539 - 2*x^535 - 2*x^

==inv3==
==invq==
0.603804826736 0.00139117240906 0.00310206413269
True
True
2*x^723 + 2*x^668 + 2*x^650 - 2*x^645 - 2*x^643 - 2*x^642 - 2*x^641 + 2*x^637 - 2*x^630 - 2*x^627 + 2*x^623 - 2*x^621 - 2*x^619 + 2*x^612 + 2*x^610 - 2*x^608 - 2*x^605 + 2*x^604 - 2*x^603 + 2*x^602 - 2*x^599 + 2*x^598 + 2*x^596 + 2*x^594 + 2*x^593 + 2*x^592 + 2*x^591 + 2*x^575 - 2*x^570 + 2*x^568 - 2*x^567 - 2*x^566 + 2*x^565 - 2*x^564 - 2*x^563 - 2*x^561 - 2*x^559 + 2*x^557 + 2*x^555 + 2*x^554 + 2*x^552 + 6*x^551 + 2*x^550 - 2*x^549 - 2*x^546 + 2*x^544 - 2*x^543 + 2*x^542 - 2*x^539 - 2*x^536 + 2*x^533 - 2*x^531 + 2*x^527 + 2*x^525 - 4*x^523 - 2*x^522 + 2*x^521 - 4*x^518 - 4*x^517 + 2*x^515 + 4*x^514 + 4*x^511 - 2*x^510 - 4*x^509 + 2*x^507 + 2*x^506 - 2*x^504 - 4*x^503 - 2*x^499 + 2*x^498 + 4*x^497 + 4*x^494 - 6*x^492 + 2*x^491 + 4*x^490 - 2*x^489 + 2*x^486 + 2*x^485 + 2*x^484 - 2*x^483 - 2*x^482 - 2*x^481 + 2*x^480 - 2*x^479 - 6*x^478 + 2*x^476 + 4*x^475 + 2*x^474 + 2*x^473 - 2*x^469 - 6*x^468 + 2*x^467 - 2*x

KeyboardInterrupt: 

In [11]:
ntru.n = 5
f = R([-1,1,0,0,1])
ntru.__inv_poly_mod3__(f)

(True, x^3 + x^2 - 1)

In [5]:
n=743
d1=11;d2=11
for i in range(100):
    a=ZZ.random_element(2*d1, n-d2)
    print a

480
397
612
334
712
477
54
44
672
149
157
612
471
390
474
326
294
575
25
507
253
123
193
328
63
187
139
444
162
267
489
594
346
669
281
111
385
640
60
332
109
252
634
162
496
274
416
713
588
239
517
370
157
286
71
359
578
677
292
333
140
252
502
162
111
216
305
515
50
324
377
579
697
27
244
699
699
469
110
455
421
290
219
365
175
460
66
243
582
698
602
71
542
493
118
234
730
576
231
690
